In [3]:
import pyautogui
import time

In [88]:
import numpy as np
import matplotlib.pyplot as plt

def takeScreen():
    im1 = pyautogui.screenshot(region=(200,260,260,100))
    mat = np.array(im1.getdata()).reshape(100, 260, 3)
    mat =  mat[:,:,2]
    return mat
    #plt.imshow(mat, cmap="gray")
    #plt.show()

def getState(sc, lastAct):
    return (sc[20,50], sc[20,225], lastAct)

def getReward():
    im1 = pyautogui.screenshot(region=(240,360,200,100))
    mat = np.array(im1.getdata()).reshape(100, 200, 3)
    mat =  mat[:,:,0]
    
    if(mat[70,157] < 100 or mat[70,41] < 100):
        return -1
    else:
        return 1

In [89]:
import win32api,win32con

def leftClick():
    win32api.mouse_event(win32con.MOUSEEVENTF_LEFTDOWN,0,0)
    time.sleep(.01)
    win32api.mouse_event(win32con.MOUSEEVENTF_LEFTUP,0,0)
    
def mousePos(cord):
    win32api.SetCursorPos((cord[0], cord[1]))

In [90]:
def dangerLeft(mat):
    if mat[20,50] < 230:
        return True
    else:
        return False

def dangerRight(mat):
    if mat[20,225] < 230:
        return True
    else:
        return False
    
def cutLeft():
    mousePos([250,600])
    leftClick()
    
def cutRight():
    mousePos([420,600])
    leftClick()

In [133]:
q = QLearn([0,1], epsilon=0.2)

In [134]:
%matplotlib inline


lastState = None
lastAct = 0

games = 0
sleepTime = 0.5

while(True):
    
    sc = takeScreen()
    rew = getReward()
    state = getState(sc, lastAct)
    
    if lastState != None:
        q.learn(lastState, lastAct, rew, state)
        
    if rew == -1:
        games =  games+1
        if games > 10:
            q.epsilon = 0
            sleepTime = 0.15
        print "lastState: "+str(lastState)+" lastAction: "+str(lastAct)+" newState: "+str(state)
        q.epsilon = q.epsilon-0.01
        mousePos((350,600))
        leftClick()
        mousePos((250, 600))
        time.sleep(2)
        lastAct = 0
        
    else:
        action = q.chooseAction(state)

        if action == 0 and lastAct == 0:
            leftClick()
        elif action == 1 and lastAct == 1:
            leftClick()
        elif action == 0 and lastAct == 1:
            cutLeft()
        elif action == 1 and lastAct == 0:
            cutRight()

        lastState = state
        lastAct = action
        #plt.imshow(sc)
        #plt.show()
        time.sleep(sleepTime)


lastState: (255, 127, 1) lastAction: 1 newState: (255, 255, 1)
lastState: (127, 255, 1) lastAction: 0 newState: (255, 255, 0)
lastState: (255, 255, 0) lastAction: 1 newState: (255, 255, 1)
lastState: (255, 127, 0) lastAction: 1 newState: (255, 255, 1)
lastState: (127, 255, 0) lastAction: 0 newState: (255, 255, 0)
lastState: (255, 255, 1) lastAction: 0 newState: (255, 255, 0)
lastState: (127, 255, 1) lastAction: 0 newState: (255, 255, 0)
lastState: (255, 255, 0) lastAction: 1 newState: (255, 255, 1)
lastState: (255, 255, 1) lastAction: 1 newState: (255, 127, 1)
lastState: (255, 255, 1) lastAction: 0 newState: (255, 255, 0)
lastState: (255, 255, 0) lastAction: 0 newState: (255, 255, 0)
lastState: (255, 127, 1) lastAction: 0 newState: (255, 127, 0)
lastState: (255, 127, 1) lastAction: 0 newState: (255, 255, 0)
lastState: (255, 240, 1) lastAction: 0 newState: (255, 255, 0)


IOError: screen grab failed

In [106]:
import pprint

pp = pprint.PrettyPrinter(indent=4)
pp.pprint(q.q)

{   ((127, 255, 1), 0): -0.276,
    ((127, 255, 1), 1): 4.533239905321846,
    ((255, 127, 1), 0): 3.720120348391027,
    ((255, 255, 0), 0): 3.035833291283843,
    ((255, 255, 0), 1): 3.5364164888960663,
    ((255, 255, 1), 1): 4.353038348589128}


In [1]:
import random


class QLearn:
    def __init__(self, actions, epsilon=0.1, alpha=0.2, gamma=0.9):
        self.q = {}

        self.epsilon = epsilon
        self.alpha = alpha
        self.gamma = gamma
        self.actions = actions

    def getQ(self, state, action):
        return self.q.get((state, action), 0.0)
        # return self.q.get((state, action), 1.0)

    def learnQ(self, state, action, reward, value):
        oldv = self.q.get((state, action), None)
        if oldv is None:
            self.q[(state, action)] = reward
        else:
            self.q[(state, action)] = oldv + self.alpha * (value - oldv)

    def chooseAction(self, state):
        if random.random() < self.epsilon:
            action = random.choice(self.actions)
        else:
            q = [self.getQ(state, a) for a in self.actions]
            maxQ = max(q)
            count = q.count(maxQ)
            if count > 1:
                best = [i for i in range(len(self.actions)) if q[i] == maxQ]
                i = random.choice(best)
            else:
                i = q.index(maxQ)

            action = self.actions[i]
        return action

    def learn(self, state1, action1, reward, state2):
        maxqnew = max([self.getQ(state2, a) for a in self.actions])
        self.learnQ(state1, action1, reward, reward + self.gamma*maxqnew)

import math
def ff(f,n):
    fs = "{:f}".format(f)
    if len(fs) < n:
        return ("{:"+n+"s}").format(fs)
    else:
        return fs[:n]